## 前馈神经网络 feedforward neuronal network:

## 1. 感知机算法：

> 通过一条直线对数据进行分类，但是无法处理非线性问题。（具体参考 week2 term1 ml）

<img src="NLP_github/perceptron.png" width="500" height="500">

## 2. 多层感知机：

> 多层感知机也称之为前馈神经网络。**（全联接网络）注意：只有对隐含层的输出使用激活函数才可以处理非线性问题。**

<img src="NLP_github/nn.png" width="400" height="400">

## 3. 前向传播与反向传播：文本分类为例

### 3.1 前向传播：

<img src="NLP_github/NN_1.png" width="500" height="500">

首先，输入X为一篇文章中每一个单词onehot编码，经过W[0]后，hidden layer（或 word embedding）的输入为 X * W[0]，将所有单词（单词数量为n）输入完毕后，得到大小为n行4列的矩阵，取均值后得到大小为1行4列的向量，用此向量表示文章。然后，文章向量经过word embedding层的激活函数作为此层的输出，同理，输出与W[1]矩阵相乘，经过输出层的softmax函数，作为最终的输出。最后，通过交叉熵函数对神经网络内的参数进行更新。

### 3.2 反向传播：


<img src="NLP_github/bp_example.jpeg" width="500" height="500">

反向传播需要链式求导，其公式如下：

$$
\frac{\partial L}{\partial e} = \frac{\partial L}{\partial c} \frac{\partial c}{\partial e}
$$

#### 1. 更新参数W[1]：
$$
\frac{\partial \text{L}}{\partial \text{W}[1]} = \frac{\partial \text{L}}{\partial \text{softmax}(y)} \frac{\partial \text{softmax}(y)}{\partial y} \frac{\partial y}{\partial \text{W}[1]} 
$$

* softmax函数求导：

>注意softmax函数求导后的结果是雅可比矩阵，如下图所示。
$$
\frac{\partial y}{\partial x} = \begin{align*}
            \begin{bmatrix}
                \frac{\partial y_1}{\partial x_1} & \frac{\partial y_2}{\partial x_1} \\
                \frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} \\
            \end{bmatrix} 
\end{align*}
$$
>
>$$
\frac{\partial y}{\partial x} = \frac{\partial }{\partial x_j} \left(\frac{exp(x_i)}{\sum_{i=1}^n exp(x_i)} \right) = \frac{exp^\prime(x_i) \left(\sum_{i=1}^n exp(x_i)\right) - exp(x_i)\left(\sum_{i=1}^n exp(x_i)\right)^\prime}{\left(\sum_{i=1}^n exp(x_i)\right)^2}
$$
>
>$$
    = \frac{exp^\prime(x_i) \left(\sum_{i=1}^n exp(x_i)\right) - exp(x_i) exp(x_j)}{\left(\sum_{i=1}^n exp(x_i)\right)^2} = \frac{exp(x_i)}{\sum_{i=1}^n exp(x_i)} - \frac{exp(x_j) exp(x_i)}{\left(\sum_{i=1}^n exp(x_i)\right)^2}
$$
* 当 i 等于 j 时，
$$
\frac{exp(x_i)}{\sum_{i=1}^n exp(x_i)} - \frac{exp(x_i) exp(x_i)}{\left(\sum_{i=1}^n exp(x_i)\right)^2} = y_i - y_i^2 = y_i(1-y_i) = P_i (1-P_i)
$$
* 当 i 不等于 j 时，
$$
\frac{0}{\sum_{i=1}^n exp(x_i)} - \frac{exp(x_i) exp(x_j)}{\left(\sum_{i=1}^n exp(x_i)\right)^2} = - y_i y_j = -P_i  P_j
$$

* **交叉熵+softmax函数求导:** 
>$$
\frac{\partial \text{L}(x_j)}{\partial x_j} = \frac{\partial \text{L}(x_j)}{\partial \text{softmax}(x_{j})} \frac{\partial \text{softmax}(x_{j})}{\partial x_j} = - \sum_{i=1}^n \frac{\partial}{\partial y_i} \left( \hat y_i \ln y_i \right) \frac{\partial y_i}{\partial x_j} = - \sum_{i=1}^n \frac{\hat y_i}{y_i} \frac{\partial y_i}{\partial x_j} = - \frac{\hat y_j}{y_j} \frac{\partial y_j}{\partial x_j} - \sum_{i \neq j} \frac{\hat y_i}{y_i} \frac{\partial y_i}{\partial x_j}
$$
$$
= - \frac{\hat y_j}{y_j} \times y_j (1-y_j) - \sum_{i \neq j} \frac{\hat y_i}{y_i} \times (-y_i y_j) = - \hat y_j (1-y_j) + \sum_{i \neq j} \hat y_i y_j = - \hat y_j + \left(\hat y_j y_j + \sum_{i \neq j} \hat y_i y_j \right) = y_j - \hat y_j
$$
其中，$\hat y_i$代表是0或1。
对交叉熵以及softmax求导后的结果为：
$$
\frac{\partial L}{\partial y}=\begin{align*}
            \begin{bmatrix}
                p_1  \\
                p_2  \\
                \vdots\\ 
                p_j - 1\\
                p_n
            \end{bmatrix} 
\end{align*}
$$

接下来，
$$
\frac{\partial y}{\partial \text{W}[1]} = h_{output} 
$$

所以W[1]的梯度为，
$$
\frac{\partial L}{\partial W[1]} = h_{output}^T \times \frac{\partial L}{\partial y}
$$

更新W[1]：
$$
W[1] = W[1] - \frac{\partial L}{\partial W[1]}
$$


**注意：提前更新每一层与最后同时更新全部均可。** 

#### 2. 更新W[0]：

$$
\frac{\partial \text{L}}{\partial \text{W}[0]} = \frac{\partial \text{L}}{\partial \text{softmax}(y)} \frac{\partial \text{softmax}(y)}{\partial y} \frac{\partial y}{\partial \text{h}_{output}} \frac{\partial \text{h}_{output}}{\partial \text{h}_{input}} \frac{\partial \text{h}_{input}}{\partial \text{W}[0]}
$$

其中，$\frac{\partial y}{\partial \text{h}_{output}}$为更新后或为更新的W[1]，$\frac{\partial \text{h}_{output}}{\partial \text{h}_{input}}$ 为激活函数求，$\frac{\partial \text{h}_{input}}{\partial \text{W}[0]}$为上一层的输出。

重复上步骤更新参数。

**反向传播伪代码：**

<img src="NLP_github/bp.png" width="500" height="500">

## 4. 防止过拟合：

* **L2-regularisation：in the weights of each layer (added in the loss function of each layer)**


* **Dropout：randomly ignore a percentage (e.g. 20% or 50%) of layer outputs during training**

## 5. 神经网络应用 - Word2vec：

**Word2vec：基于分布假设（经常具有相同相邻字词的字词往往在语义上相似）**
>
>两种模型：
* **skip-gram model：中心词来预测周围词。**
* **continuous BOW (CBOW)：周围词来预测中心词。**
>
>下图为两种模型的结构，层之间全联接，注意：有两个权重矩阵，其中输入与隐含层之间的代表word embedding。
<img src="NLP_github/word2vec_2.png" width="500" height="500">
>
>两种性能提升方法：
* **negative sampling**
* **hierarchical softmax**


### 5.1 skip-gram 模型: 中心词来预测周围词

下图为模型的输入与输出。其中，蓝色表示输入（中心词），使用（输入，**输出（one-hot）**）作为数据集。**注意：输出为one-hot，相应位置的神经元类别为1其余为0，二分类问题。**
<img src="NLP_github/word2vec_1.png" width="500" height="500">

例如：“the man loves his son.”，当中心词（输入）为“loves”时，其输出分别为：the，man，his，son。（输入，输出）可以表达成（loves，the）（loves，man）（loves，his）（loves，son），通过最大化输出概率训练模型。

$$
\max \text{P}(the, man, his, son|loves) = \max \text{P}(the|loves)P(man|loves)P(his|loves)P(son|loves)
$$

其中，条件概率通过softmax函数计算，每个单词的向量为对应输入层到隐含层矩阵的某一行（向量点乘的几何意义是夹角用来表示两个单词的相似度）。
$$
\text{P}(output_j|input) = \frac{\exp(\vec{output_j} \times \vec{input})}{\sum_{i=1}^n \exp(\vec{output_i} \times \vec {input})}
$$

$$
Loss = \sum_{i=1}^n P_i^y \times (1-P_i)^y
$$

### 5.2 CBOW模型：用周围词来预测中心词

原理与skip-gra相似，**周围词求和作为隐含层的输出**，然后二分类问题根据中心词的one-hot作为标签。

### 5.3 负采样：

在训练神经网络时，如上述前向反向传播公式推导一样，每输入一个样本会导致所有参数被更新，当参数非常多时，效率会非常缓慢。通过负采样，每次仅更新一小部分参数，会极大降低计算量。

负采样每次仅更新positive word以及5到20个negative word对应的参数（one-hot对应为1的为positive，其余为negative）。一个单词被选做negative的概率由它的词频决定，具体概率公式如下：

$$
\text{P}(w_i) = \frac{f(word_j)^{\frac{3}{4}}}{\sum_{i=1}^n f(word_i)^{\frac{3}{4}}}
$$



### 5.4 hierarchical softmax - 哈夫曼树（压缩算法） 

如下图所示，有7个字母，如果每个字母使用onehot编码的话，每个字母用长度为7的向量表示，但是采用了哈弗曼编码，每个字母的向量长度将会大大缩小，并且出现频率高的字母的编码长度会很小由于靠近根节点。
<img src="NLP_github/htree_1.png" width="400" height="400">

在word2cev模型中，哈夫曼树用于隐含层到输出层，如下图所示。
<img src="NLP_github/htree.png" width="500" height="500">
其中，哈夫曼树内的除叶子结点之外的每个结点内包含一个参数向量，其长度等于输入的长度，通过更新这些节点的参数对模型进行训练，例如下图所示。
<img src="NLP_github/htree_2.png" width="400" height="400">

* softmax+交叉熵：https://zhuanlan.zhihu.com/p/37740860

* Skip-Gram介绍：https://zhuanlan.zhihu.com/p/27234078

* Skip-Gram与GBOW比较： https://zhuanlan.zhihu.com/p/37477611

* 反向传播参数更新：http://colah.github.io/posts/2015-08-Backprop/ 

* 哈夫曼树：https://blog.csdn.net/zynash2/article/details/81636338